In [ ]:
!nvidia-smi

Mon Nov 14 00:37:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip install evaluate
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.8 MB/s 
     |████████████████████████████████| 163 kB 44.0 MB/s 
     |████████████████████████████████| 7.6 MB 51.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 4.6 MB/s 
     |████████████████████████████████| 212 kB 62.4 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 115 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 48.2 MB/s 
     |████████████████████████████████| 115 kB 56.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully unin

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
from evaluate import load

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# raw_datasets = load_dataset("wmt16", "de-en")
raw_datasets = load_dataset("wmt16", "de-en")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
metric = load("bertscore")

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 4548885
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2169
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2999
    })
})


In [ ]:
raw_dataset_train = load_dataset("wmt16", "de-en", split="train[:20%]")
raw_dataset_test = load_dataset("wmt16", "de-en", split="test[:20%]")
raw_dataset_validation = load_dataset("wmt16", "de-en", split="validation[:20%]")

In [ ]:
print(raw_dataset_train)
print(raw_dataset_test)
print(raw_dataset_validation)

Dataset({
    features: ['translation'],
    num_rows: 909777
})
Dataset({
    features: ['translation'],
    num_rows: 600
})
Dataset({
    features: ['translation'],
    num_rows: 434
})


In [ ]:
from datasets import DatasetDict 
raw_datasets = DatasetDict({'train': raw_dataset_train, 'test': raw_dataset_test, 'validation': raw_dataset_validation})

In [ ]:
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 909777
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 600
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 434
    })
})


In [ ]:
predictions = ["hello there", "general kenobi hello"]
references = ["hello there", "general kenobi there"]
results = metric.compute(predictions=predictions, references=references, lang='en')['f1']
print(results)

[1.0, 0.9645148515701294]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
prefix = "translate English to German: "

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# from datasets import load_from_disk
# tokenized_datasets = load_from_disk('./tokenizer')

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/910 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(tokenized_datasets['train'][0])

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}, 'input_ids': [13959, 1566, 12, 2968, 10, 419, 4078, 102, 1575, 13, 8, 2363, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [15158, 24860, 74, 11216, 425, 7, 4267, 32, 221, 1, 0, 0, 0, 0, 

In [ ]:
print(tokenizer.decode(tokenized_datasets['train'][0]['input_ids'], skip_special_tokens=True))

translate English to German: Resumption of the session


In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/d78aea13fa7ecd06c29e3e46195d6341255065d5/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_b

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch import nn
from torch.distributions import Categorical
import sys, torch

class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bleu = []
        self.distributions = []
        self.counter = 0 

    def compute_loss(self, model, inputs, return_outputs=False):
        log_probs = []
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        attn_mask = torch.where(labels != -100, inputs['attention_mask'], 0)
        masked_labels = labels*attn_mask
        decoded_labels = tokenizer.batch_decode(masked_labels,skip_special_tokens=True)
        # decoded_labels = [[i] for i in decoded_labels]

        attn_mask = torch.unsqueeze(attn_mask, 2)
        masked_logits = logits*attn_mask
        decoded_logits = []
        log_probs_batch = []
        cat = Categorical(logits=masked_logits)
        samplebatch = cat.sample()
        log_probs_batch = cat.log_prob(samplebatch)
        log_probs_batch = torch.mean(log_probs_batch,1)
        decoded_logits = tokenizer.batch_decode(samplebatch, skip_special_tokens=True)

        bert_scores = metric.compute(predictions=decoded_logits, references=decoded_labels, lang="de")['f1']
        loss = sum([-log_prob*bert_score for log_prob, bert_score in zip(log_probs_batch, bert_scores)])
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    fp16=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 909777
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 56862
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/cf732291d5a8eace7b973ccd13c95ec07b19e734/pytorch_model.bin
All the weights of BertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-500
Configuration saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-500/config.json
Model weights saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-1000
Configuration saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-1000/config.json
Model weights saved in ./gdrive/MyDrive/11785-Final/t5-20-reinforce-bertscore/results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./gdrive/MyDrive/11785-Final/t

RuntimeError: ignored

In [ ]:
metric = load('sacrebleu')

In [ ]:
from tqdm.notebook import tqdm
import torch
scores = 0
for data in tqdm(tokenized_datasets["validation"]):
  label = data["translation"]["de"]
  input = tokenizer("translate English to German: "+data["translation"]['en'],return_tensors="pt").input_ids.to("cuda")
  model = model.cpu()
  input = input.cpu()
  output = model.generate(torch.LongTensor(input))
  tr = tokenizer.decode(output[0],skip_special_tokens=True)
  bert_score = metric.compute(predictions=[tr], references=[[label]])['score']
  scores+=bert_score

scores /= len(tokenized_datasets["validation"])
print(scores)

  0%|          | 0/434 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


15.16423070709688
